<a href="https://colab.research.google.com/github/ch23s020/Assignment3/blob/main/Assignment3_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import gdown
import random
import time
import pandas as pd
import csv

Data laod and  Pre-Processing

In [ ]:
# Download dataset from Google Drive

url = 'https://drive.google.com/uc?id=1A3M8MiTtdCPQlfAgJBOpZlG5QOX8b7Tz'

output = '/content/transliteration_data.csv'

gdown.download(url, output, quiet=False)


# Load and preprocess dataset from CSV file
# Separated the Training, Validation and Test Data for Marathi.

def load_data(file_path):

    data = []

    with open(file_path, 'r', encoding='utf-8') as csvfile:

        csvreader = csv.reader(csvfile)

        # Trying to remove the Index error while passing the separated csv file.
        for idx, row in enumerate(csvreader):

            try:
                x = str(row[0])

                y = str(row[1])

                data.append((x, y))

            except IndexError:   # To locate the issue

                print(f"IndexError in row {idx + 1}: {row}")
    return data

data = load_data(output)



# Data Preparation

class TransliterationDataset(Dataset):

    def __init__(self, data, char2index, max_length=20):

        self.data = data

        self.max_length = max_length

        self.char2index = char2index



    def __len__(self):

        return len(self.data)

    def __getitem__(self, idx):

        x, y = self.data[idx]

        x_indices = [self.char2index[c] for c in x] + [self.char2index['<PAD>']] * (self.max_length - len(x))

        y_indices = [self.char2index[c] for c in y] + [self.char2index['<PAD>']] * (self.max_length - len(y))

        return torch.tensor(x_indices), torch.tensor(y_indices), len(x), len(y)

def collate_fn(batch):

    x, y, x_lengths, y_lengths = zip(*batch)

    x = torch.nn.utils.rnn.pad_sequence(x, batch_first=True, padding_value=char2index['<PAD>'])

    y = torch.nn.utils.rnn.pad_sequence(y, batch_first=True, padding_value=char2index['<PAD>'])

    return x, y


RNN


In [ ]:
class EncoderRNN(pass):

class DecoderRNN(pass):

class Seq2Seq(nn.Module):



Training
